In [133]:
# 🚀 Milvus를 사용한 완전한 RAG 파이프라인 구현
# 이 노트북은 다음 기능들을 포함합니다:
# 1. 마크다운 파일 읽기
# 2. 텍스트 청킹 (헤더 기반 + 문장 기반)
# 3. 임베딩 생성 (OpenAI 또는 테스트용 임베딩)
# 4. Milvus 벡터 데이터베이스 저장
# 5. 유사도 검색
# 6. 대화형 검색 인터페이스

print("🚀 완전한 RAG 파이프라인이 준비되었습니다!")
print("아래 셀들을 순서대로 실행하세요:")
print("1. 라이브러리 임포트")
print("2. 파일 읽기")
print("3. 텍스트 청킹")
print("4. 임베딩 생성")
print("5. Milvus 연결 및 설정")
print("6. 데이터 저장")
print("7. 검색 테스트")
print("8. 대화형 검색")

🚀 완전한 RAG 파이프라인이 준비되었습니다!
아래 셀들을 순서대로 실행하세요:
1. 라이브러리 임포트
2. 파일 읽기
3. 텍스트 청킹
4. 임베딩 생성
5. Milvus 연결 및 설정
6. 데이터 저장
7. 검색 테스트
8. 대화형 검색


In [136]:
# 필요한 라이브러리 설치 및 임포트
import os
import re
import numpy as np
from typing import List, Dict, Any
from pymilvus import connections, Collection, FieldSchema, CollectionSchema, DataType, utility
from openai import OpenAI
from tqdm import tqdm
import tiktoken

from dotenv import load_dotenv

load_dotenv()

# 환경 변수 설정 (API 키가 필요한 경우)
# os.environ["OPENAI_API_KEY"] = "your-openai-api-key"

print("라이브러리 임포트 완료!")


라이브러리 임포트 완료!


In [137]:
# 1. 마크다운 파일 읽기
with open('ceca9bf3-6183-40bc-ad64-52911b2015b3.md', 'r', encoding='utf-8') as file:
    content = file.read()

print(f"문서 전체 길이: {len(content)} 문자")
print("=" * 50)
print("문서 미리보기:")
print(content[:200] + "...")


문서 전체 길이: 3093 문자
문서 미리보기:
# 「AX브릿지위원회」 소개자료(벤처기업협회 회원소통본부, 2024.10.11.)

데이터, 네트워크, AI 등 첨단 디지털 기술 분야 선도벤처와 유망 스
타트업이 함께 모여 디지털시대의 경쟁우위 확보를 위해 필요한 전략
적 협력을 추구하고 규제 정비와 정책적 지원을 정부에 건의하여 기
업환경 개선을 도모

# □ 위원회 구성○ 데이터, 네트워크, AI 등...


In [138]:
# 2. 텍스트 청킹 함수 정의
def clean_text(text: str) -> str:
    """텍스트 전처리"""
    # 여러 줄바꿈을 단일 줄바꿈으로 변경
    text = re.sub(r'\n+', '\n', text)
    # 여러 공백을 단일 공백으로 변경
    text = re.sub(r'\s+', ' ', text)
    # 앞뒤 공백 제거
    text = text.strip()
    return text

def chunk_by_sentences(text: str, max_chunk_size: int = 500, overlap: int = 50) -> List[str]:
    """문장 단위로 텍스트를 청킹"""
    # 문장 구분자로 분리
    sentences = re.split(r'[.!?]\s+|[\n]+', text)
    sentences = [s.strip() for s in sentences if s.strip()]
    
    chunks = []
    current_chunk = ""
    
    for sentence in sentences:
        # 현재 청크에 문장을 추가했을 때의 길이 확인
        test_chunk = current_chunk + " " + sentence if current_chunk else sentence
        
        if len(test_chunk) <= max_chunk_size:
            current_chunk = test_chunk
        else:
            # 현재 청크를 저장하고 새 청크 시작
            if current_chunk:
                chunks.append(current_chunk.strip())
            current_chunk = sentence
    
    # 마지막 청크 추가
    if current_chunk:
        chunks.append(current_chunk.strip())
    
    return chunks

def chunk_by_headers(text: str, max_chunk_size: int = 800) -> List[Dict[str, Any]]:
    """헤더 기반으로 텍스트를 청킹하고 메타데이터 포함"""
    # 헤더 패턴 찾기
    lines = text.split('\n')
    chunks = []
    current_chunk = ""
    current_header = ""
    
    for line in lines:
        line = line.strip()
        if not line:
            continue
            
        # 헤더 확인 (# 시작하는 라인)
        if line.startswith('#'):
            # 이전 청크가 있으면 저장
            if current_chunk:
                chunk_data = {
                    'content': clean_text(current_chunk),
                    'header': current_header,
                    'length': len(current_chunk)
                }
                chunks.append(chunk_data)
            
            # 새 청크 시작
            current_header = line
            current_chunk = line + "\n"
        else:
            current_chunk += line + "\n"
            
            # 청크 크기 확인
            if len(current_chunk) > max_chunk_size:
                # 문장 단위로 세분화
                sub_chunks = chunk_by_sentences(current_chunk, max_chunk_size // 2)
                for sub_chunk in sub_chunks:
                    chunk_data = {
                        'content': clean_text(sub_chunk),
                        'header': current_header,
                        'length': len(sub_chunk)
                    }
                    chunks.append(chunk_data)
                current_chunk = ""
    
    # 마지막 청크 추가
    if current_chunk:
        chunk_data = {
            'content': clean_text(current_chunk),
            'header': current_header,
            'length': len(current_chunk)
        }
        chunks.append(chunk_data)
    
    return chunks

# 텍스트 청킹 실행
chunks = chunk_by_headers(content)
print(f"생성된 청크 수: {len(chunks)}")
print("=" * 50)

# 첫 번째 청크 확인
for i, chunk in enumerate(chunks[:3]):
    print(f"청크 {i+1}:")
    print(f"헤더: {chunk['header']}")
    print(f"길이: {chunk['length']} 문자")
    print(f"내용: {chunk['content'][:100]}...")
    print("-" * 30)


생성된 청크 수: 12
청크 1:
헤더: # 「AX브릿지위원회」 소개자료(벤처기업협회 회원소통본부, 2024.10.11.)
길이: 171 문자
내용: # 「AX브릿지위원회」 소개자료(벤처기업협회 회원소통본부, 2024.10.11.) 데이터, 네트워크, AI 등 첨단 디지털 기술 분야 선도벤처와 유망 스 타트업이 함께 모여 디지털...
------------------------------
청크 2:
헤더: # □ 위원회 구성○ 데이터, 네트워크, AI 등 첨단 기술분야 선도 벤처·스타트업으로
길이: 297 문자
내용: # □ 위원회 구성○ 데이터, 네트워크, AI 등 첨단 기술분야 선도 벤처·스타트업으로 초기 멤버를 구성한 후 위원회 결정에 따라 규모를 확정 * AX(AI transformati...
------------------------------
청크 3:
헤더: # □ 운영조직- ○ 의 장 : 이주완 대표(메가존클라우드)
길이: 85 문자
내용: # □ 운영조직- ○ 의 장 : 이주완 대표(메가존클라우드) - ○ 운영위원 : 10명 내외(추가섭외중) - ○ 사 무 국 : 벤처기업협회 회원소통본부...
------------------------------


In [139]:
# 3. 임베딩 생성 함수 (OpenAI 사용)
class EmbeddingGenerator:
    def __init__(self, model_name="text-embedding-3-small"):
        # OpenAI API 키가 설정되어 있다고 가정
        # 실제 사용시에는 환경변수나 .env 파일로 설정하세요
        self.client = OpenAI()  # api_key는 환경변수에서 자동 로드
        self.model_name = model_name
        self.dimension = 1536  # text-embedding-3-small의 차원
    
    def get_embedding(self, text: str) -> List[float]:
        """단일 텍스트의 임베딩 생성"""
        try:
            response = self.client.embeddings.create(
                input=text,
                model=self.model_name
            )
            return response.data[0].embedding
        except Exception as e:
            print(f"임베딩 생성 오류: {e}")
            # 오류 발생시 랜덤 벡터 반환 (테스트용)
            return np.random.rand(self.dimension).tolist()
    
    def get_embeddings_batch(self, texts: List[str], batch_size: int = 10) -> List[List[float]]:
        """배치로 임베딩 생성"""
        embeddings = []
        for i in tqdm(range(0, len(texts), batch_size), desc="임베딩 생성"):
            batch = texts[i:i+batch_size]
            batch_embeddings = []
            for text in batch:
                embedding = self.get_embedding(text)
                batch_embeddings.append(embedding)
            embeddings.extend(batch_embeddings)
        return embeddings

# 테스트용 간단한 임베딩 생성기 (OpenAI API 키가 없는 경우)
class SimpleEmbeddingGenerator:
    def __init__(self, dimension=768):
        self.dimension = dimension
        # 간단한 해시 기반 임베딩 생성 (실제 운영에서는 사용하지 마세요)
        
    def get_embedding(self, text: str) -> List[float]:
        """텍스트 기반 간단한 임베딩 생성 (테스트용)"""
        # 실제로는 transformer 모델이나 OpenAI API를 사용해야 합니다
        np.random.seed(hash(text) % 2**32)  # 동일한 텍스트는 동일한 임베딩
        embedding = np.random.rand(self.dimension).astype(np.float32)
        # 정규화
        embedding = embedding / np.linalg.norm(embedding)
        return embedding.tolist()
    
    def get_embeddings_batch(self, texts: List[str]) -> List[List[float]]:
        return [self.get_embedding(text) for text in tqdm(texts, desc="임베딩 생성")]

# 임베딩 생성기 초기화 (OpenAI API 키가 있으면 EmbeddingGenerator, 없으면 SimpleEmbeddingGenerator 사용)
try:
    embedding_generator = EmbeddingGenerator()
    print("OpenAI 임베딩 생성기 초기화 완료")
except:
    embedding_generator = SimpleEmbeddingGenerator()
    print("테스트용 임베딩 생성기 초기화 완료 (실제 운영시에는 OpenAI나 다른 임베딩 모델을 사용하세요)")


OpenAI 임베딩 생성기 초기화 완료


In [140]:
# 4. 청크들의 임베딩 생성
chunk_texts = [chunk['content'] for chunk in chunks]
print(f"임베딩을 생성할 청크 수: {len(chunk_texts)}")

# 임베딩 생성
embeddings = embedding_generator.get_embeddings_batch(chunk_texts)
print(f"생성된 임베딩 수: {len(embeddings)}")
print(f"임베딩 차원: {len(embeddings[0]) if embeddings else 0}")

# 청크에 임베딩 정보 추가
for i, chunk in enumerate(chunks):
    chunk['embedding'] = embeddings[i]
    chunk['id'] = i

print("임베딩 생성 완료!")


임베딩을 생성할 청크 수: 12


임베딩 생성: 100%|██████████| 2/2 [00:04<00:00,  2.46s/it]

생성된 임베딩 수: 12
임베딩 차원: 1536
임베딩 생성 완료!


In [118]:
print(chunks)

[{'content': '# 「AX브릿지위원회」 소개자료(벤처기업협회 회원소통본부, 2024.10.11.) 데이터, 네트워크, AI 등 첨단 디지털 기술 분야 선도벤처와 유망 스 타트업이 함께 모여 디지털시대의 경쟁우위 확보를 위해 필요한 전략 적 협력을 추구하고 규제 정비와 정책적 지원을 정부에 건의하여 기 업환경 개선을 도모', 'header': '# 「AX브릿지위원회」 소개자료(벤처기업협회 회원소통본부, 2024.10.11.)', 'length': 171, 'embedding': [0.008927841670811176, -0.024381255730986595, 0.07859369367361069, 0.051678698509931564, 0.03169563040137291, 0.012142821215093136, -0.040611520409584045, 0.05158308520913124, 0.0075832875445485115, -0.021990936249494553, 0.042189132422208786, -0.044507741928100586, -0.005151137709617615, -0.05206115171313286, 0.005936955101788044, -0.002704048529267311, -0.011383894830942154, 0.001967531396076083, -0.01828594133257866, 0.0076729245483875275, -0.03860365226864815, -0.02787112072110176, -0.007905980572104454, 0.015823911875486374, 0.006573377642780542, -0.030882922932505608, -0.014329963363707066, -0.01596733182668686, 0.028205765411257744, -0.05449927598237991, 0.016110751777887344, -0.040587618947029114, -0.00712315

In [ ]:
# 5. Milvus 연결 및 컬렉션 설정
def setup_milvus_collection():
    """Milvus 컬렉션 설정"""
    # Milvus 연결 (docker-compose로 실행된 Milvus 서버)
    connections.connect("default", host="localhost", port="19530")
    print("Milvus 연결 완료")
    
    # 컬렉션명
    collection_name = "new_document_test"
    
    # 기존 컬렉션이 있으면 삭제
    if utility.has_collection(collection_name):
        utility.drop_collection(collection_name)
        print(f"기존 컬렉션 '{collection_name}' 삭제")
    
    # 스키마 정의
    fields = [
        FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=False),
        FieldSchema(name="content", dtype=DataType.VARCHAR, max_length=2000),
        FieldSchema(name="header", dtype=DataType.VARCHAR,max_length=2000),
        FieldSchema(name="length", dtype=DataType.INT64),
        FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=len(embeddings[0]))
    ]
    
    schema = CollectionSchema(fields, f"{collection_name} collection")
    collection = Collection(collection_name, schema)
    
    # 인덱스 생성 (벡터 검색을 위한)
    index_params = {
        "metric_type": "L2",  # 또는 "IP" (Inner Product), "COSINE"
        "index_type": "IVF_FLAT",
        "params": {"nlist": 128}
    }
    collection.create_index("embedding", index_params)
    print("인덱스 생성 완료")
    
    return collection

try:
    collection = setup_milvus_collection()
    print("Milvus 컬렉션 설정 완료")
except Exception as e:
    print(f"Milvus 연결 오류: {e}")
    print("docker-compose up -d 명령으로 Milvus를 먼저 실행해주세요")


Milvus 연결 완료
인덱스 생성 완료
Milvus 컬렉션 설정 완료


In [142]:
# 6. 데이터를 Milvus에 저장
def insert_data_to_milvus(collection, chunks):
    """청크 데이터를 Milvus에 삽입"""

    data = []
    ids = []
    contents = []
    headers = []
    lengths = []
    embeddings = []

    for chunk in chunks:
        ids.append(chunk['id'])
        contents.append(chunk['content'])
        headers.append(chunk['header'])
        lengths.append(chunk['length'])
        embeddings.append(chunk['embedding'])

    data = [ids, contents, headers, lengths, embeddings]
    
    # 데이터 삽입
    collection.insert(data)
    collection.flush()  # 즉시 저장
    
    print(f"{len(data)}개 청크를 Milvus에 저장 완료")
    
try:
    insert_data_to_milvus(collection, chunks)
    
    # 컬렉션 로드 (검색을 위해 필요)
    collection.load()
    print("컬렉션 로드 완료 - 검색 준비됨")
    
    # 저장된 데이터 수 확인
    print(f"저장된 총 엔티티 수: {collection.num_entities}")
    
except Exception as e:
    print(f"데이터 저장 오류: {e}")


5개 청크를 Milvus에 저장 완료
컬렉션 로드 완료 - 검색 준비됨
저장된 총 엔티티 수: 12


In [143]:
# 7. 검색 함수 정의
def search_documents(query: str, top_k: int = 5) -> List[Dict[str, Any]]:
    """문서에서 유사한 청크 검색"""
    # 쿼리 임베딩 생성
    query_embedding = embedding_generator.get_embedding(query)
    
    # 검색 파라미터
    search_params = {
        "metric_type": "L2",
        "params": {"nprobe": 10}
    }
    
    # 검색 실행
    results = collection.search(
        data=[query_embedding],
        anns_field="embedding",
        param=search_params,
        limit=top_k,
        output_fields=["content", "header", "length"]
    )
    
    # 결과 정리
    search_results = []
    for hits in results:
        for hit in hits:
            result = {
                'id': hit.id,
                'score': hit.score,  # 거리 점수 (L2의 경우 낮을수록 유사)
                'content': hit.entity.get('content'),
                'header': hit.entity.get('header'),
                'length': hit.entity.get('length')
            }
            search_results.append(result)
    
    return search_results

def display_search_results(query: str, results: List[Dict[str, Any]]):
    """검색 결과를 보기 좋게 출력"""
    print(f"🔍 검색 쿼리: '{query}'")
    print(f"📊 검색 결과: {len(results)}개")
    print("=" * 80)
    
    for i, result in enumerate(results, 1):
        print(f"📄 결과 {i} (ID: {result['id']}, 점수: {result['score']:.4f})")
        print(f"🏷️  헤더: {result['header']}")
        print(f"📝 내용: {result['content'][:200]}...")
        print(f"📏 길이: {result['length']} 문자")
        print("-" * 80)


In [144]:
# 8. 검색 테스트
try:
    # 다양한 검색 쿼리 테스트
    test_queries = [
        "AX브릿지위원회 의장은 누구인가?",
        "메가존클라우드 대표이사",
        "정기포럼은 언제 열리나?",
        "AI 관련 기업들",
        "운영위원 명단"
    ]
    
    for query in test_queries:
        print("\n" + "="*100)
        results = search_documents(query, top_k=3)
        display_search_results(query, results)
        
except Exception as e:
    print(f"검색 오류: {e}")
    print("Milvus 연결을 확인해주세요")



🔍 검색 쿼리: 'AX브릿지위원회 의장은 누구인가?'
📊 검색 결과: 3개
📄 결과 1 (ID: 4, 점수: 1.0374)
🏷️  헤더: # □ 추진내용- ○ 2024. 4. 15. 킥오프회의 개최 (운영방안, 명칭 논의)
📝 내용: # □ 추진내용- ○ 2024. 4. 15. 킥오프회의 개최 (운영방안, 명칭 논의) - ○ 2024. 4. ~ 5. 위원회 멤버 영입 - ○ 2024. 7. 1. AX브릿지위원회 출범식 및 출범기념포럼 개최 - ○ 2024. 5. ~ 12. 정기포럼 개최, AX사례발굴...
📏 길이: 152 문자
--------------------------------------------------------------------------------
📄 결과 2 (ID: 11, 점수: 1.1234)
🏷️  헤더: # □ 운영위원| 순 서 | 소속/성명 | 사진 | 주요약력 |
📝 내용: | 10 | 다올티에스 (주) 홍정화 대표이사 | ![image](/image/placeholder) | <운영위원> ○ 前 한국오라클 마케팅본부장 前 NetApp 지사장 前 EMC SW사업총괄본부장 다올티에스 초대CEO | | 11 | (주)아사달 서창녕 대표이사 | ![image](/image/placeholder) | <운영위원> ○ 서울대학교 경제학...
📏 길이: 439 문자
--------------------------------------------------------------------------------
📄 결과 3 (ID: 9, 점수: 1.1426)
🏷️  헤더: # □ 운영위원| 순 서 | 소속/성명 | 사진 | 주요약력 |
📝 내용: | <운영위원> ○ 현 세븐미어캣 대표이사 ○ 전 카페24 대외협력팀장 ○ 전 아데코코리아 창업수석 ○ 과학기술정보통신부 장관상 ○ 대통령직속청년위원회 표창상 | | 순서 | 소속/성명 | 사진 | 주요약력 | | --- | --- | --- | --- | | 8 | (주)스파이어 테크놀로지 강군

In [145]:
# 9. 인터랙티브 검색 함수
def interactive_search():
    """대화형 검색 인터페이스"""
    print("🔍 문서 검색 시스템")
    print("검색을 종료하려면 'quit' 또는 'exit'를 입력하세요")
    print("-" * 50)
    
    while True:
        query = input("\n검색어를 입력하세요: ").strip()
        
        if query.lower() in ['quit', 'exit', '종료']:
            print("검색을 종료합니다.")
            break
        
        if not query:
            print("검색어를 입력해주세요.")
            continue
        
        try:
            results = search_documents(query, top_k=3)
            display_search_results(query, results)
        except Exception as e:
            print(f"검색 중 오류 발생: {e}")

# 사용자가 원할 때 실행할 수 있도록 준비
print("interactive_search() 함수를 호출하여 대화형 검색을 시작할 수 있습니다.")


interactive_search() 함수를 호출하여 대화형 검색을 시작할 수 있습니다.


In [146]:
# 10. 추가 유틸리티 함수들

def get_collection_stats():
    """컬렉션 통계 정보 확인"""
    try:
        print("📊 Milvus 컬렉션 통계:")
        print(f"- 총 엔티티 수: {collection.num_entities}")
        print(f"- 컬렉션명: {collection.name}")
        print(f"- 스키마: {collection.schema}")
        
        # 인덱스 정보
        indexes = collection.indexes
        print(f"- 인덱스 수: {len(indexes)}")
        for idx in indexes:
            print(f"  * {idx.field_name}: {idx.params}")
            
    except Exception as e:
        print(f"통계 정보 조회 오류: {e}")

def cleanup_milvus():
    """Milvus 리소스 정리"""
    try:
        collection.drop()
        connections.disconnect("default")
        print("Milvus 리소스 정리 완료")
    except Exception as e:
        print(f"정리 중 오류: {e}")

# 컬렉션 정보 확인
try:
    get_collection_stats()
except:
    print("컬렉션 통계를 가져올 수 없습니다. Milvus 연결을 확인해주세요.")


📊 Milvus 컬렉션 통계:
- 총 엔티티 수: 12
- 컬렉션명: new_document_test
- 스키마: {'auto_id': False, 'description': 'new_document_test collection', 'fields': [{'name': 'id', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': False}, {'name': 'content', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 2000}}, {'name': 'header', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 2000}}, {'name': 'length', 'description': '', 'type': <DataType.INT64: 5>}, {'name': 'embedding', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 1536}}], 'enable_dynamic_field': False}
- 인덱스 수: 1
  * embedding: {'metric_type': 'L2', 'index_type': 'IVF_FLAT', 'params': {'nlist': 128}}


In [147]:
# 📄 기존 컬렉션에 새로운 문서 추가하기

def add_new_document_to_collection(file_path: str, collection_name: str = "new_document_test"):
    """기존 컬렉션에 새로운 문서를 추가하는 함수"""
    
    # 1. 기존 컬렉션 연결 및 로드
    try:
        if not utility.has_collection(collection_name):
            print(f"❌ 컬렉션 '{collection_name}'이 존재하지 않습니다.")
            return False
        
        collection = Collection(collection_name)
        collection.load()
        
        # 기존 데이터의 최대 ID 확인 (새 ID 생성을 위해)
        query_result = collection.query(
            expr="id >= 0",
            output_fields=["id"],
            limit=16384  # 최대 개수
        )
        
        if query_result:
            max_id = max([item['id'] for item in query_result])
            next_id = max_id + 1
        else:
            next_id = 0
            
        print(f"📊 기존 데이터 수: {len(query_result)}개")
        print(f"🆔 새 문서 시작 ID: {next_id}")
        
    except Exception as e:
        print(f"❌ 컬렉션 연결 오류: {e}")
        return False
    
    # 2. 새 문서 읽기
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            new_content = file.read()
        print(f"📖 새 문서 읽기 완료: {file_path}")
        print(f"📏 문서 길이: {len(new_content)} 문자")
        
    except Exception as e:
        print(f"❌ 파일 읽기 오류: {e}")
        return False
    
    # 3. 새 문서 청킹
    new_chunks = chunk_by_headers(new_content, max_chunk_size=800)
    print(f"✂️ 생성된 청크 수: {len(new_chunks)}")
    
    # 4. 새 청크들에 ID 할당
    for i, chunk in enumerate(new_chunks):
        chunk['id'] = next_id + i
        chunk['source_file'] = file_path  # 출처 파일 정보 추가
    
    # 5. 임베딩 생성
    print("🔄 임베딩 생성 중...")
    new_chunk_texts = [chunk['content'] for chunk in new_chunks]
    new_embeddings = embedding_generator.get_embeddings_batch(new_chunk_texts)
    
    # 임베딩을 청크에 추가
    for i, chunk in enumerate(new_chunks):
        chunk['embedding'] = new_embeddings[i]
    
    # 6. 새 데이터를 컬렉션에 추가
    try:
        new_data = []
        ids = []
        contetns = []
        headers = []
        lengths = []
        embeddings = []
        for chunk in new_chunks:
            ids.append(chunk['id'])
            contetns.append(chunk['content'])
            headers.append(chunk['header'])
            lengths.append(chunk['length'])
            embeddings.append(chunk['embedding'])

        new_data = [ids, contetns, headers, lengths, embeddings]
        
        # 데이터 삽입
        collection.insert(new_data)
        collection.flush()
        print(f"✅ {len(new_data)}개의 새 청크를 컬렉션에 추가했습니다!")
        
        # 최종 통계
        print(f"📊 총 엔티티 수: {collection.num_entities}")
        
        return True
        
    except Exception as e:
        print(f"❌ 데이터 삽입 오류: {e}")
        return False

# 함수 사용 예시
print("💡 사용 방법:")
print("add_new_document_to_collection('새파일.md')")
print("\n또는 현재 파일을 다시 추가하려면:")
print("add_new_document_to_collection('ceca9bf3-6183-40bc-ad64-52911b2015b3.md')")

💡 사용 방법:
add_new_document_to_collection('새파일.md')

또는 현재 파일을 다시 추가하려면:
add_new_document_to_collection('ceca9bf3-6183-40bc-ad64-52911b2015b3.md')


In [148]:
# 🧪 새 문서 추가 테스트

# 현재 선택한 파일을 기존 컬렉션에 추가 (테스트용으로 같은 파일 재추가)
# 실제로는 다른 마크다운 파일을 추가하시면 됩니다.

try:
    # 기존 컬렉션 상태 확인
    print("📋 추가 전 컬렉션 상태:")
    get_collection_stats()
    print()
    
    # 새 문서 추가 (예시로 같은 파일을 다시 추가)
    success = add_new_document_to_collection('9f9e37a6-55f4-43c4-8285-b4b976f5dd4b.md')
    
    if success:
        print("\n✅ 문서 추가 완료!")
        print("\n📋 추가 후 컬렉션 상태:")
        get_collection_stats()
    else:
        print("\n❌ 문서 추가 실패")
        
except Exception as e:
    print(f"❌ 오류 발생: {e}")
    print("Milvus 연결 상태를 확인해주세요.")


📋 추가 전 컬렉션 상태:
📊 Milvus 컬렉션 통계:
- 총 엔티티 수: 12
- 컬렉션명: new_document_test
- 스키마: {'auto_id': False, 'description': 'new_document_test collection', 'fields': [{'name': 'id', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': False}, {'name': 'content', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 2000}}, {'name': 'header', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 2000}}, {'name': 'length', 'description': '', 'type': <DataType.INT64: 5>}, {'name': 'embedding', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 1536}}], 'enable_dynamic_field': False}
- 인덱스 수: 1
  * embedding: {'metric_type': 'L2', 'index_type': 'IVF_FLAT', 'params': {'nlist': 128}}

📊 기존 데이터 수: 12개
🆔 새 문서 시작 ID: 12
📖 새 문서 읽기 완료: 9f9e37a6-55f4-43c4-8285-b4b976f5dd4b.md
📏 문서 길이: 15431 문자
✂️ 생성된 청크 수: 87
🔄 임베딩 생성 중...


임베딩 생성: 100%|██████████| 9/9 [00:30<00:00,  3.38s/it]


✅ 5개의 새 청크를 컬렉션에 추가했습니다!
📊 총 엔티티 수: 99

✅ 문서 추가 완료!

📋 추가 후 컬렉션 상태:
📊 Milvus 컬렉션 통계:
- 총 엔티티 수: 99
- 컬렉션명: new_document_test
- 스키마: {'auto_id': False, 'description': 'new_document_test collection', 'fields': [{'name': 'id', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': False}, {'name': 'content', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 2000}}, {'name': 'header', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 2000}}, {'name': 'length', 'description': '', 'type': <DataType.INT64: 5>}, {'name': 'embedding', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 1536}}], 'enable_dynamic_field': False}
- 인덱스 수: 1
  * embedding: {'metric_type': 'L2', 'index_type': 'IVF_FLAT', 'params': {'nlist': 128}}


In [150]:
# 🔍 확장된 검색 테스트 (새 문서 포함)

def enhanced_search_with_source_info(query: str, top_k: int = 5):
    """소스 정보를 포함한 향상된 검색"""
    # 쿼리 임베딩 생성
    query_embedding = embedding_generator.get_embedding(query)
    
    # 검색 파라미터
    search_params = {
        "metric_type": "L2",
        "params": {"nprobe": 10}
    }
    
    # 검색 실행
    results = collection.search(
        data=[query_embedding],
        anns_field="embedding",
        param=search_params,
        limit=top_k,
        output_fields=["content", "header", "length"]
    )
    
    # 결과 정리
    search_results = []
    for hits in results:
        for hit in hits:
            result = {
                'id': hit.id,
                'score': hit.score,
                'content': hit.entity.get('content'),
                'header': hit.entity.get('header'), 
                'length': hit.entity.get('length'),
                'is_new_document': hit.id >= 12  # 기존 12개 이후는 새 문서
            }
            search_results.append(result)
    
    return search_results

def display_enhanced_search_results(query: str, results):
    """향상된 검색 결과 표시"""
    print(f"🔍 검색 쿼리: '{query}'")
    print(f"📊 검색 결과: {len(results)}개")
    print("=" * 80)
    
    for i, result in enumerate(results, 1):
        source_indicator = "🆕 [새 문서]" if result['is_new_document'] else "📄 [기존 문서]"
        print(f"{source_indicator} 결과 {i} (ID: {result['id']}, 점수: {result['score']:.4f})")
        print(f"🏷️  헤더: {result['header']}")
        print(f"📝 내용: {result['content'][:200]}...")
        print(f"📏 길이: {result['length']} 문자")
        print("-" * 80)

# 확장된 검색 테스트
try:
    print("🧪 확장된 검색 테스트 (기존 + 새 문서 포함)")
    
    test_queries = [
        "AX브릿지위원회 의장",
        "메가존클라우드",
        "정기포럼",
        "AI 기업", 
        "운영위원",
        "서울 든든 급식"
    ]
    
    for query in test_queries:
        print("\n" + "="*100)
        results = enhanced_search_with_source_info(query, top_k=5)
        display_enhanced_search_results(query, results)
        
except Exception as e:
    print(f"❌ 검색 오류: {e}")
    print("컬렉션 로드 상태를 확인해주세요.")


🧪 확장된 검색 테스트 (기존 + 새 문서 포함)

🔍 검색 쿼리: 'AX브릿지위원회 의장'
📊 검색 결과: 5개
📄 [기존 문서] 결과 1 (ID: 4, 점수: 1.0432)
🏷️  헤더: # □ 추진내용- ○ 2024. 4. 15. 킥오프회의 개최 (운영방안, 명칭 논의)
📝 내용: # □ 추진내용- ○ 2024. 4. 15. 킥오프회의 개최 (운영방안, 명칭 논의) - ○ 2024. 4. ~ 5. 위원회 멤버 영입 - ○ 2024. 7. 1. AX브릿지위원회 출범식 및 출범기념포럼 개최 - ○ 2024. 5. ~ 12. 정기포럼 개최, AX사례발굴...
📏 길이: 152 문자
--------------------------------------------------------------------------------
📄 [기존 문서] 결과 2 (ID: 0, 점수: 1.1450)
🏷️  헤더: # 「AX브릿지위원회」 소개자료(벤처기업협회 회원소통본부, 2024.10.11.)
📝 내용: # 「AX브릿지위원회」 소개자료(벤처기업협회 회원소통본부, 2024.10.11.) 데이터, 네트워크, AI 등 첨단 디지털 기술 분야 선도벤처와 유망 스 타트업이 함께 모여 디지털시대의 경쟁우위 확보를 위해 필요한 전략 적 협력을 추구하고 규제 정비와 정책적 지원을 정부에 건의하여 기 업환경 개선을 도모...
📏 길이: 171 문자
--------------------------------------------------------------------------------
📄 [기존 문서] 결과 3 (ID: 9, 점수: 1.1547)
🏷️  헤더: # □ 운영위원| 순 서 | 소속/성명 | 사진 | 주요약력 |
📝 내용: | <운영위원> ○ 현 세븐미어캣 대표이사 ○ 전 카페24 대외협력팀장 ○ 전 아데코코리아 창업수석 ○ 과학기술정보통신부 장관상 ○ 대통령직속청년위원회 표창상 | | 순서 | 소속/성명 | 사진 | 주요약력 | | --- | --- | --- | --- 

In [128]:
# 🛠️ 추가 유틸리티 함수들

def list_all_documents_in_collection():
    """컬렉션 내 모든 문서 ID와 헤더 목록 조회"""
    try:
        # 모든 데이터 조회
        all_data = collection.query(
            expr="id >= 0",
            output_fields=["id", "header", "length"],
            limit=16384
        )
        
        print(f"📚 컬렉션 내 총 {len(all_data)}개 문서 청크:")
        print("-" * 80)
        
        for item in sorted(all_data, key=lambda x: x['id']):
            source_type = "🆕 [새 문서]" if item['id'] >= 12 else "📄 [기존 문서]"
            print(f"{source_type} ID: {item['id']}, 길이: {item['length']}, 헤더: {item['header'][:50]}...")
            
    except Exception as e:
        print(f"❌ 문서 목록 조회 오류: {e}")

def delete_documents_by_id_range(start_id: int, end_id: int):
    """ID 범위로 문서 삭제"""
    try:
        expr = f"id >= {start_id} and id <= {end_id}"
        collection.delete(expr)
        collection.flush()
        print(f"✅ ID {start_id}~{end_id} 범위의 문서들을 삭제했습니다.")
        print(f"📊 현재 엔티티 수: {collection.num_entities}")
        
    except Exception as e:
        print(f"❌ 문서 삭제 오류: {e}")

def search_by_specific_document_type(query: str, include_new_docs: bool = True, include_original_docs: bool = True):
    """특정 문서 타입으로 검색 범위 제한"""
    results = enhanced_search_with_source_info(query, top_k=10)
    
    # 필터링
    filtered_results = []
    for result in results:
        is_new = result['is_new_document']
        if (is_new and include_new_docs) or (not is_new and include_original_docs):
            filtered_results.append(result)
    
    return filtered_results[:5]  # 상위 5개만 반환

print("🛠️ 추가 유틸리티 함수들이 준비되었습니다:")
print("- list_all_documents_in_collection(): 모든 문서 목록 조회")
print("- delete_documents_by_id_range(start, end): ID 범위로 문서 삭제")
print("- search_by_specific_document_type(query, include_new, include_original): 문서 타입별 검색")


🛠️ 추가 유틸리티 함수들이 준비되었습니다:
- list_all_documents_in_collection(): 모든 문서 목록 조회
- delete_documents_by_id_range(start, end): ID 범위로 문서 삭제
- search_by_specific_document_type(query, include_new, include_original): 문서 타입별 검색


In [129]:
# 📈 현재 시스템 용량 확인 및 모니터링

def check_system_capacity():
    """현재 Milvus 시스템의 용량 사용 현황 확인"""
    try:
        # 연결 정보
        print("🔗 Milvus 연결 정보:")
        print(f"- 호스트: localhost:19530")
        
        # 컬렉션 목록 조회
        collection_list = utility.list_collections()
        print(f"\n📚 전체 컬렉션 수: {len(collection_list)}")
        print(f"- 최대 허용 컬렉션 수: 65,536개")
        print(f"- 사용률: {len(collection_list)/65536*100:.2f}%")
        
        # 각 컬렉션별 상세 정보
        print(f"\n📋 컬렉션별 상세 정보:")
        print("-" * 80)
        
        total_entities = 0
        for collection_name in collection_list:
            try:
                coll = Collection(collection_name)
                if utility.has_collection(collection_name):
                    # 컬렉션이 로드되어 있는지 확인
                    try:
                        coll.load()  # 로드되지 않은 경우 로드 시도
                        entity_count = coll.num_entities
                        total_entities += entity_count
                        
                        # 컬렉션 설정 정보
                        schema = coll.schema
                        fields_count = len(schema.fields)
                        
                        print(f"📄 {collection_name}:")
                        print(f"   - 엔티티 수: {entity_count:,}개")
                        print(f"   - 필드 수: {fields_count}개 (최대: 64개)")
                        
                        # 벡터 필드 정보
                        for field in schema.fields:
                            if field.dtype == DataType.FLOAT_VECTOR:
                                print(f"   - 벡터 차원: {field.params.get('dim', 'N/A')}차원 (최대: 32,768차원)")
                        
                    except Exception as e:
                        print(f"📄 {collection_name}: 로드 불가 - {str(e)[:50]}...")
            except Exception as e:
                print(f"📄 {collection_name}: 조회 오류 - {str(e)[:50]}...")
                
        print("-" * 80)
        print(f"🎯 총 엔티티 수: {total_entities:,}개 (제한: 무제한)")
        
        # 메모리 사용량 추정 (대략적)
        avg_vector_size = 1536 * 4  # 1536차원 * 4바이트(float32)
        estimated_memory_mb = total_entities * avg_vector_size / (1024 * 1024)
        print(f"📊 추정 메모리 사용량: {estimated_memory_mb:.1f} MB")
        
        # 권장사항
        print(f"\n💡 권장사항:")
        if len(collection_list) > 1000:
            print("⚠️  컬렉션 수가 많습니다. 성능 모니터링을 권장합니다.")
        if total_entities > 1000000:
            print("⚠️  대용량 데이터입니다. 메모리 사용량을 모니터링하세요.")
        if len(collection_list) < 10 and total_entities < 100000:
            print("✅ 현재 사용량이 적절합니다. 추가 확장 가능합니다.")
            
    except Exception as e:
        print(f"❌ 시스템 용량 확인 오류: {e}")

def estimate_capacity_for_new_data(document_count: int, avg_chunk_size: int = 800, chunks_per_doc: int = 10):
    """새 데이터 추가시 용량 예측"""
    print(f"📊 새 데이터 용량 예측:")
    print(f"- 추가 예정 문서 수: {document_count:,}개")
    print(f"- 문서당 평균 청크 수: {chunks_per_doc}개")
    print(f"- 청크당 평균 크기: {avg_chunk_size} 문자")
    
    total_chunks = document_count * chunks_per_doc
    print(f"- 예상 총 청크 수: {total_chunks:,}개")
    
    # 메모리 사용량 추정
    vector_size = 1536 * 4  # 1536차원 float32
    text_size = avg_chunk_size * 1  # 대략적인 텍스트 크기
    total_size_mb = total_chunks * (vector_size + text_size) / (1024 * 1024)
    
    print(f"- 예상 메모리 사용량: {total_size_mb:.1f} MB")
    print(f"- 예상 디스크 사용량: {total_size_mb * 1.5:.1f} MB (인덱스 포함)")

# 현재 시스템 용량 확인
print("🔍 현재 Milvus 시스템 용량 분석:")
try:
    check_system_capacity()
except:
    print("시스템 용량 확인 불가 - Milvus 연결을 확인하세요.")

print("\n" + "="*80)

# 새 데이터 용량 예측 예시
print("🔮 새 데이터 추가시 용량 예측 (예시):")
estimate_capacity_for_new_data(
    document_count=1000,      # 1,000개 문서
    avg_chunk_size=800,       # 청크당 800자
    chunks_per_doc=10         # 문서당 10개 청크
)


🔍 현재 Milvus 시스템 용량 분석:
🔗 Milvus 연결 정보:
- 호스트: localhost:19530

📚 전체 컬렉션 수: 1
- 최대 허용 컬렉션 수: 65,536개
- 사용률: 0.00%

📋 컬렉션별 상세 정보:
--------------------------------------------------------------------------------
📄 document_chunks:
   - 엔티티 수: 24개
   - 필드 수: 5개 (최대: 64개)
   - 벡터 차원: 1536차원 (최대: 32,768차원)
--------------------------------------------------------------------------------
🎯 총 엔티티 수: 24개 (제한: 무제한)
📊 추정 메모리 사용량: 0.1 MB

💡 권장사항:
✅ 현재 사용량이 적절합니다. 추가 확장 가능합니다.

🔮 새 데이터 추가시 용량 예측 (예시):
📊 새 데이터 용량 예측:
- 추가 예정 문서 수: 1,000개
- 문서당 평균 청크 수: 10개
- 청크당 평균 크기: 800 문자
- 예상 총 청크 수: 10,000개
- 예상 메모리 사용량: 66.2 MB
- 예상 디스크 사용량: 99.3 MB (인덱스 포함)


In [130]:
# 💻 파티션 분할 구현 예제

def create_partitioned_collection():
    """파티션을 사용하는 컬렉션 생성 예제"""
    from pymilvus import Collection, FieldSchema, CollectionSchema, DataType, Partition
    
    collection_name = "document_chunks_partitioned"
    
    # 기존 컬렉션이 있으면 삭제
    if utility.has_collection(collection_name):
        utility.drop_collection(collection_name)
        print(f"기존 컬렉션 '{collection_name}' 삭제")
    
    # 스키마 정의 (기존과 동일하지만 partition_key 추가 고려)
    fields = [
        FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=False),
        FieldSchema(name="content", dtype=DataType.VARCHAR, max_length=2000),
        FieldSchema(name="header", dtype=DataType.VARCHAR, max_length=500),
        FieldSchema(name="category", dtype=DataType.VARCHAR, max_length=100),  # 파티션 키로 사용
        FieldSchema(name="date", dtype=DataType.VARCHAR, max_length=50),
        FieldSchema(name="length", dtype=DataType.INT64),
        FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=768)  # 간단한 예제용
    ]
    
    schema = CollectionSchema(fields, f"{collection_name} with partitions")
    collection = Collection(collection_name, schema)
    
    # 파티션 생성
    partitions = [
        "tech_docs",      # 기술 문서
        "news_articles",  # 뉴스 기사  
        "research_papers", # 연구 논문
        "company_docs",   # 회사 문서
        "other_docs"      # 기타 문서
    ]
    
    for partition_name in partitions:
        if not collection.has_partition(partition_name):
            collection.create_partition(partition_name)
            print(f"✅ 파티션 '{partition_name}' 생성")
    
    # 인덱스 생성
    index_params = {
        "metric_type": "L2",
        "index_type": "IVF_FLAT", 
        "params": {"nlist": 128}
    }
    collection.create_index("embedding", index_params)
    
    return collection

def insert_data_to_partitions(collection, sample_data):
    """파티션별로 데이터 삽입"""
    # 카테고리별로 데이터 분할
    partition_data = {
        "tech_docs": [],
        "news_articles": [],
        "research_papers": [],
        "company_docs": [],
        "other_docs": []
    }
    
    # 샘플 데이터 생성 (실제로는 문서에서 추출)
    for i, data in enumerate(sample_data):
        category = data.get('category', 'other_docs')
        
        # 데이터 포맷 맞추기
        row = [
            data['id'],
            data['content'][:2000],
            data['header'][:500], 
            category,
            data.get('date', '2024-01-01'),
            data['length'],
            data['embedding']
        ]
        
        if category in partition_data:
            partition_data[category].append(row)
        else:
            partition_data["other_docs"].append(row)
    
    # 각 파티션에 데이터 삽입
    for partition_name, data_list in partition_data.items():
        if data_list:  # 데이터가 있는 경우만
            collection.insert(data_list, partition_name=partition_name)
            print(f"📄 '{partition_name}' 파티션에 {len(data_list)}개 데이터 삽입")
    
    collection.flush()
    return partition_data

def search_in_specific_partition(collection, query_embedding, partition_names=None, top_k=5):
    """특정 파티션에서만 검색"""
    search_params = {
        "metric_type": "L2",
        "params": {"nprobe": 10}
    }
    
    # 컬렉션 로드
    collection.load()
    
    # 파티션 지정 검색
    results = collection.search(
        data=[query_embedding],
        anns_field="embedding",
        param=search_params,
        limit=top_k,
        partition_names=partition_names,  # 특정 파티션만 검색
        output_fields=["content", "header", "category", "date"]
    )
    
    return results

print("🛠️ 파티션 기반 컬렉션 생성 예제:")
print("create_partitioned_collection() - 파티션 컬렉션 생성")
print("insert_data_to_partitions() - 파티션별 데이터 삽입")  
print("search_in_specific_partition() - 특정 파티션 검색")


🛠️ 파티션 기반 컬렉션 생성 예제:
create_partitioned_collection() - 파티션 컬렉션 생성
insert_data_to_partitions() - 파티션별 데이터 삽입
search_in_specific_partition() - 특정 파티션 검색


In [131]:
# 🧪 파티션 사용 예제 실행

def demo_partition_usage():
    """파티션 사용 데모"""
    try:
        print("📁 파티션 기반 컬렉션 생성 중...")
        
        # 파티션 컬렉션 생성
        partitioned_collection = create_partitioned_collection()
        
        # 샘플 데이터 준비 (기존 청크 데이터를 카테고리별로 분류)
        sample_data = []
        
        # 기존 청크 데이터가 있다면 활용
        if 'chunks' in globals() and chunks:
            for i, chunk in enumerate(chunks[:10]):  # 예제로 10개만 사용
                # 헤더 내용에 따라 카테고리 자동 분류
                header = chunk.get('header', '').lower()
                
                if 'ax' in header or '브릿지' in header or '위원회' in header:
                    category = 'company_docs'
                elif '운영' in header or '조직' in header:
                    category = 'company_docs'
                elif '기술' in header or 'ai' in header:
                    category = 'tech_docs'
                else:
                    category = 'other_docs'
                
                # 간단한 임베딩 생성 (실제로는 기존 임베딩 사용)
                simple_embedding = np.random.rand(768).astype(np.float32).tolist()
                
                sample_data.append({
                    'id': i + 1000,  # ID 충돌 방지
                    'content': chunk['content'],
                    'header': chunk['header'],
                    'category': category,
                    'date': '2024-01-01',
                    'length': chunk['length'],
                    'embedding': simple_embedding
                })
        else:
            # 기존 데이터가 없는 경우 샘플 데이터 생성
            categories = ['tech_docs', 'company_docs', 'news_articles', 'research_papers']
            for i in range(20):
                category = categories[i % len(categories)]
                sample_data.append({
                    'id': i + 1000,
                    'content': f'샘플 {category} 내용 {i}',
                    'header': f'# {category} 헤더 {i}',
                    'category': category,
                    'date': '2024-01-01',
                    'length': 100,
                    'embedding': np.random.rand(768).astype(np.float32).tolist()
                })
        
        # 파티션별 데이터 삽입
        print("\n📤 파티션별 데이터 삽입 중...")
        partition_data = insert_data_to_partitions(partitioned_collection, sample_data)
        
        # 파티션별 통계 출력
        print(f"\n📊 파티션별 데이터 분포:")
        for partition_name, data_list in partition_data.items():
            if data_list:
                print(f"  - {partition_name}: {len(data_list)}개 문서")
        
        # 파티션 목록 확인
        partitions = partitioned_collection.partitions
        print(f"\n📁 생성된 파티션 목록:")
        for partition in partitions:
            print(f"  - {partition.name}")
        
        # 특정 파티션에서만 검색 예제
        print(f"\n🔍 파티션별 검색 예제:")
        
        if sample_data:
            query_embedding = sample_data[0]['embedding']
            
            # 전체 검색
            print("1. 전체 컬렉션 검색:")
            all_results = search_in_specific_partition(
                partitioned_collection, 
                query_embedding, 
                partition_names=None, 
                top_k=3
            )
            for hits in all_results:
                print(f"  전체 검색 결과: {len(hits)}개")
            
            # 특정 파티션만 검색  
            print("2. 'company_docs' 파티션만 검색:")
            company_results = search_in_specific_partition(
                partitioned_collection,
                query_embedding,
                partition_names=['company_docs'],
                top_k=3
            )
            for hits in company_results:
                print(f"  회사 문서 검색 결과: {len(hits)}개")
                
        print(f"\n✅ 파티션 데모 완료!")
        return partitioned_collection
        
    except Exception as e:
        print(f"❌ 파티션 데모 오류: {e}")
        return None

# 파티션 데모 실행
print("🎯 파티션 사용 예제를 실행하려면 demo_partition_usage()를 호출하세요!")
print("\n파티션의 주요 장점:")
print("✅ 검색 성능 향상 - 관련 파티션만 스캔")
print("✅ 메모리 효율성 - 필요한 파티션만 로드") 
print("✅ 데이터 관리 - 카테고리별 독립적 관리")
print("✅ 병렬 처리 - 여러 파티션 동시 처리")


🎯 파티션 사용 예제를 실행하려면 demo_partition_usage()를 호출하세요!

파티션의 주요 장점:
✅ 검색 성능 향상 - 관련 파티션만 스캔
✅ 메모리 효율성 - 필요한 파티션만 로드
✅ 데이터 관리 - 카테고리별 독립적 관리
✅ 병렬 처리 - 여러 파티션 동시 처리


In [132]:
# 🚀 실제 클러스터 확장 구현 가이드

def create_cluster_config():
    """클러스터 확장용 Docker Compose 파일 생성"""
    
    cluster_compose = """
version: '3.8'

networks:
  milvus:
    driver: bridge

services:
  etcd:
    container_name: milvus-etcd
    image: quay.io/coreos/etcd:v3.5.5
    environment:
      - ETCD_AUTO_COMPACTION_MODE=revision
      - ETCD_AUTO_COMPACTION_RETENTION=1000
      - ETCD_QUOTA_BACKEND_BYTES=4294967296
      - ETCD_SNAPSHOT_COUNT=50000
    volumes:
      - ./volumes/etcd:/etcd
    command: etcd -advertise-client-urls=http://127.0.0.1:2379 -listen-client-urls http://0.0.0.0:2379 --data-dir /etcd
    healthcheck:
      test: ["CMD", "etcdctl", "endpoint", "health"]
      interval: 30s
      timeout: 20s
      retries: 3
    networks:
      - milvus

  minio:
    container_name: milvus-minio
    image: minio/minio:RELEASE.2023-03-20T20-16-18Z
    environment:
      MINIO_ACCESS_KEY: minioadmin
      MINIO_SECRET_KEY: minioadmin
    ports:
      - "9001:9001"
      - "9000:9000"
    volumes:
      - ./volumes/minio:/minio_data
    command: minio server /minio_data --console-address ":9001"
    healthcheck:
      test: ["CMD", "curl", "-f", "http://localhost:9000/minio/health/live"]
      interval: 30s
      timeout: 20s
      retries: 3
    networks:
      - milvus

  # === 코디네이터 서비스 ===
  rootcoord:
    container_name: milvus-rootcoord
    image: milvusdb/milvus:v2.6.x-latest
    command: ["milvus", "run", "rootcoord"]
    environment:
      ETCD_ENDPOINTS: etcd:2379
      MINIO_ADDRESS: minio:9000
    volumes:
      - ./configs:/milvus/configs/
    depends_on:
      - "etcd"
      - "minio"
    networks:
      - milvus

  datacoord:
    container_name: milvus-datacoord
    image: milvusdb/milvus:v2.6.x-latest
    command: ["milvus", "run", "datacoord"]
    environment:
      ETCD_ENDPOINTS: etcd:2379
      MINIO_ADDRESS: minio:9000
    volumes:
      - ./configs:/milvus/configs/
    depends_on:
      - "etcd"
      - "minio"
      - "rootcoord"
    networks:
      - milvus

  querycoord:
    container_name: milvus-querycoord
    image: milvusdb/milvus:v2.6.x-latest
    command: ["milvus", "run", "querycoord"]
    environment:
      ETCD_ENDPOINTS: etcd:2379
      MINIO_ADDRESS: minio:9000
    volumes:
      - ./configs:/milvus/configs/
    depends_on:
      - "etcd"
      - "minio"
      - "rootcoord"
    networks:
      - milvus

  indexcoord:
    container_name: milvus-indexcoord
    image: milvusdb/milvus:v2.6.x-latest
    command: ["milvus", "run", "indexcoord"]
    environment:
      ETCD_ENDPOINTS: etcd:2379
      MINIO_ADDRESS: minio:9000
    volumes:
      - ./configs:/milvus/configs/
    depends_on:
      - "etcd"
      - "minio"
      - "rootcoord"
    networks:
      - milvus

  # === 워커 노드들 (확장 가능) ===
  datanode1:
    container_name: milvus-datanode-1
    image: milvusdb/milvus:v2.6.x-latest
    command: ["milvus", "run", "datanode"]
    environment:
      ETCD_ENDPOINTS: etcd:2379
      MINIO_ADDRESS: minio:9000
    volumes:
      - ./configs:/milvus/configs/
    depends_on:
      - "etcd"
      - "minio"
      - "datacoord"
    deploy:
      resources:
        limits:
          cpus: '2.0'
          memory: 4G
    networks:
      - milvus

  datanode2:
    container_name: milvus-datanode-2
    image: milvusdb/milvus:v2.6.x-latest
    command: ["milvus", "run", "datanode"]
    environment:
      ETCD_ENDPOINTS: etcd:2379
      MINIO_ADDRESS: minio:9000
    volumes:
      - ./configs:/milvus/configs/
    depends_on:
      - "etcd"
      - "minio"
      - "datacoord"
    deploy:
      resources:
        limits:
          cpus: '2.0'
          memory: 4G
    networks:
      - milvus

  querynode1:
    container_name: milvus-querynode-1
    image: milvusdb/milvus:v2.6.x-latest
    command: ["milvus", "run", "querynode"]
    environment:
      ETCD_ENDPOINTS: etcd:2379
      MINIO_ADDRESS: minio:9000
    volumes:
      - ./configs:/milvus/configs/
    depends_on:
      - "etcd"
      - "minio"
      - "querycoord"
    deploy:
      resources:
        limits:
          cpus: '4.0'
          memory: 8G
    networks:
      - milvus

  querynode2:
    container_name: milvus-querynode-2
    image: milvusdb/milvus:v2.6.x-latest
    command: ["milvus", "run", "querynode"]
    environment:
      ETCD_ENDPOINTS: etcd:2379
      MINIO_ADDRESS: minio:9000
    volumes:
      - ./configs:/milvus/configs/
    depends_on:
      - "etcd"
      - "minio"
      - "querycoord"
    deploy:
      resources:
        limits:
          cpus: '4.0'
          memory: 8G
    networks:
      - milvus

  indexnode1:
    container_name: milvus-indexnode-1
    image: milvusdb/milvus:v2.6.x-latest
    command: ["milvus", "run", "indexnode"]
    environment:
      ETCD_ENDPOINTS: etcd:2379
      MINIO_ADDRESS: minio:9000
    volumes:
      - ./configs:/milvus/configs/
    depends_on:
      - "etcd"
      - "minio"
      - "indexcoord"
    deploy:
      resources:
        limits:
          cpus: '2.0'
          memory: 4G
    networks:
      - milvus

  # === 프록시 (로드 밸런서) ===
  proxy1:
    container_name: milvus-proxy-1
    image: milvusdb/milvus:v2.6.x-latest
    command: ["milvus", "run", "proxy"]
    environment:
      ETCD_ENDPOINTS: etcd:2379
      MINIO_ADDRESS: minio:9000
    ports:
      - "19530:19530"
      - "9091:9091"
    volumes:
      - ./configs:/milvus/configs/
    depends_on:
      - "etcd"
      - "minio"
      - "rootcoord"
      - "querycoord"
      - "datacoord"
      - "indexcoord"
    networks:
      - milvus

  proxy2:
    container_name: milvus-proxy-2
    image: milvusdb/milvus:v2.6.x-latest
    command: ["milvus", "run", "proxy"]
    environment:
      ETCD_ENDPOINTS: etcd:2379
      MINIO_ADDRESS: minio:9000
    ports:
      - "19531:19530"
      - "9092:9091"
    volumes:
      - ./configs:/milvus/configs/
    depends_on:
      - "etcd"
      - "minio"
      - "rootcoord"
      - "querycoord"
      - "datacoord"
      - "indexcoord"
    networks:
      - milvus
"""

    # 파일로 저장
    with open('docker-compose-cluster.yml', 'w', encoding='utf-8') as f:
        f.write(cluster_compose)
    
    print("✅ 클러스터 Docker Compose 파일 생성: docker-compose-cluster.yml")
    
    # 설정 파일도 생성
    cluster_config = """
# Milvus 클러스터 설정 파일
etcd:
  endpoints:
    - etcd:2379

minio:
  address: minio
  port: 9000
  accessKeyID: minioadmin
  secretAccessKey: minioadmin
  bucketName: milvus-bucket

rootCoord:
  address: rootcoord
  port: 53100

queryCoord:
  address: querycoord
  port: 19531

dataCoord:
  address: datacoord
  port: 13333

indexCoord:
  address: indexcoord
  port: 31000

proxy:
  port: 19530

log:
  level: info
  file:
    rootPath: /var/lib/milvus/logs

common:
  retentionDuration: 432000  # 5 days
  entityExpiration: -1       # never expires
"""

    import os
    os.makedirs('configs', exist_ok=True)
    with open('configs/milvus.yaml', 'w', encoding='utf-8') as f:
        f.write(cluster_config)
    
    print("✅ 클러스터 설정 파일 생성: configs/milvus.yaml")

def scale_cluster_commands():
    """클러스터 확장 명령어 가이드"""
    
    commands = """
🚀 클러스터 확장 실행 단계:

1️⃣ 현재 단일 노드 중지:
   docker-compose down

2️⃣ 클러스터 모드로 시작:
   docker-compose -f docker-compose-cluster.yml up -d

3️⃣ 특정 노드만 확장 (예: querynode 추가):
   docker-compose -f docker-compose-cluster.yml up -d --scale querynode=4

4️⃣ 실시간 노드 추가 (무중단):
   docker-compose -f docker-compose-cluster.yml up -d querynode3

5️⃣ 클러스터 상태 확인:
   docker-compose -f docker-compose-cluster.yml ps

6️⃣ 로그 모니터링:
   docker-compose -f docker-compose-cluster.yml logs -f proxy1

7️⃣ 리소스 사용량 모니터링:
   docker stats

📊 성능 모니터링:
   - Milvus WebUI: http://localhost:9091
   - MinIO Console: http://localhost:9001
"""
    
    print(commands)

print("🏗️ 클러스터 확장 구현 도구:")
print("create_cluster_config() - 클러스터 설정 파일 생성")
print("scale_cluster_commands() - 확장 명령어 가이드")
print("\n💡 사용 순서:")
print("1. create_cluster_config() 실행")
print("2. scale_cluster_commands() 확인") 
print("3. 제공된 명령어로 클러스터 실행")


🏗️ 클러스터 확장 구현 도구:
create_cluster_config() - 클러스터 설정 파일 생성
scale_cluster_commands() - 확장 명령어 가이드

💡 사용 순서:
1. create_cluster_config() 실행
2. scale_cluster_commands() 확인
3. 제공된 명령어로 클러스터 실행
